In [1]:
!pip install -q pandas tqdm 
!pip install -q transformers==4.55.0 # llm requires >=4.46.0
!pip install -q safetensors==0.4.3 # downgrade for torch 2.1.0
!pip install -q bitsandbytes==0.43.2 accelerate==1.9.0 # quantization
!pip install -q peft==0.17.0 trl==0.21.0 # finetune
!pip install -q datasets


[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os, pandas as pd, json, random
from datasets import Dataset, concatenate_datasets
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer, SFTConfig
import torch
import re

In [3]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())
    
    question = " ".join(q_lines)
    return question, options

# 프롬프트 생성기
def make_prompt_auto(row):
    Question = str(row["Question"]).strip()
    Answer = str(row["Answer"]).split("답변:")[-1].strip()
    if is_multiple_choice(Question):
        question, options = extract_question_and_choices(Question)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                f"질문: {Question}\n\n"
                "답변:"
                )
    response = Answer

    return prompt, response

In [4]:
# 데이터 로드

dfs = []
dfs.append(pd.read_csv("../data/CyberMetric/mcqa.csv"))
dfs.append(pd.read_csv("../data/FinShibainu/mcqa.csv"))
dfs.append(pd.read_csv("../data/FinShibainu/qa.csv"))
dfs.append(pd.read_csv("../data/SecBench/mcqa.csv"))
dfs.append(pd.read_csv("../data/SecBench/qa.csv"))

full = pd.concat(dfs, ignore_index=True)

In [5]:
full

,Question,Answer
0,다음 중 정보의 비밀을 나타내는 것은 무엇입니까?\n1. 가용성\n2. 인증\n3....,답변: 4
1,"어떤 유형의 인증은 당신이 아는 것, 당신이 가진 것, 당신이있는 것과 같은 여러 ...",답변: 3
2,발가락은 무엇을 의미합니까?\n1. 평가 목표\n2. 평가 시간\n3. 평가 유형\...,답변: 1
3,시스템이 리소스에 대한 액세스를 요청하는 사용자가 실제로 자신이 주장하는 사람인지 ...,답변: 4
4,"기밀성, 무결성 및 데이터 및 자산의 가용성에 대한 확인 및 보증을 포함하여 정보 ...",답변: 2
...,...,...
100508,블록 체인 기술에서 거래 검증 가능성을 유지하면서 거래 개인 정보를 향상시키기 위해...,답변: 거래 개인 정보 보호를 향상시키기 위해 제로 지식 증거를 사용하는 방법은 당...
100509,Linux 시스템에서 실제 권한 에스컬레이션 취약성 (예 : CVE-2019-186...,답변: CVE-2019-18634는 Linux 시스템에 영향을 미치는 권한 에스컬레...
100510,PHP에서 문자열을 정의하기 위해 이중 인용문과 단일 따옴표 사용의 차이점은 무엇입니까?,답변: PHP에서 이중 인용문으로 정의 된 문자열은 변수를 구문 분석하는 반면 단일...
100511,PHP 코드를 식별하기 위해 PHP에 구문 분석 태그를 작성하는 방법은 무엇입니까?,"답변: PHP에서 구문 분석 태그를 작성하는 세 가지 방법, 즉 '<? php?>'..."


In [6]:
# 튜플을 딕셔너리로 변환
tuples = [make_prompt_auto(r) for _, r in full.iterrows()]
records = [{"prompt": prompt, "completion": response} for prompt, response in tuples] # SFTTrainer 에서 completion 필드 사용
random.seed(42)
random.shuffle(records)

# 간단 split
n = int(len(records)*0.95)
train_ds = Dataset.from_list(records[:n])
eval_ds  = Dataset.from_list(records[n:])

In [7]:
import unicodedata, re

ZWS = "\u200b"          # zero-width space
NBSP = "\xa0"           # non-breaking space

def clean_text(s: str) -> str:
    s = s.replace("\r\n","\n").replace("\r","\n")   # 개행 통일
    s = s.replace(NBSP," ").replace(ZWS,"")         # 이상 공백 제거
    s = unicodedata.normalize("NFKC", s)            # 유니코드 정규화
    # 문자열에 직접 박아둔 특수토큰은 제거(토크나이저에 맡김)
    for tok in ("<s>","</s>","<bos>","</eos>","<<SYS>>","<<USER>>","<<ASSISTANT>>"):
        s = s.replace(tok,"")
    return s

def normalize_pair(ex):
    p = clean_text(ex["prompt"]).rstrip()           # 뒤 공백/개행 제거
    c = clean_text(ex["completion"]).lstrip()       # 앞 공백 제거
    if not p.endswith("\n"):                        # 경계 개행 1개 강제
        p += "\n"
    return {"prompt": p, "completion": c}

train_ds = train_ds.map(normalize_pair, num_proc=4)
eval_ds  = eval_ds.map(normalize_pair,  num_proc=4)


Map (num_proc=4):   0%|          | 0/95487 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5026 [00:00<?, ? examples/s]

In [8]:
print(train_ds, eval_ds)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 95487
}) Dataset({
    features: ['prompt', 'completion'],
    num_rows: 5026
})


In [9]:
# 모델 선택
models = [
    "gemma-ko-7b", # baseline
    "ax-4.0-light-7b", # skt
    # "polyglot-12.8b",
    # "koalpaca-polyglot-12.8b",
    "midm-2.0-11.5b", # kt
    # "HyperCLOVAX-SEED-Think-14B", # naver
    # "kanana-1.5-15.7b-a3b-instruct", # kakao
    # "exaone-4.0-32b" # lg
]
selected_model = models[1]
model_path = f"/workspace/models/{selected_model}" # 로컬 저장 모델 경로

# 4bit 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16, # NaN 방지
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# # 8bit 설정
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,          # 4bit → 8bit
#     llm_int8_threshold=6.0,     # 기본값 (필요 시 조정)
#     llm_int8_has_fp16_weight=False  # True로 하면 일부 레이어 FP16 유지
# )

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

# 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto", # GPU 자동 배정
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    # trust_remote_code=True # naver
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# QLoRA Setting

base_model = prepare_model_for_kbit_training(base_model)

# LoRA (Low-Rank Adaptation) 설정
peft_config = LoraConfig(
    # LoRA 핵심 파라미터
    r=16,                       # 랭크(rank): LoRA 행렬의 차원, 낮을수록 파라미터 적음, 높을수록 표현력 증가
                                # 일반적으로 8~64 사용, 16은 성능과 효율성의 균형점
    
    lora_alpha=32,              # 스케일링 파라미터: LoRA 업데이트의 강도 조절
                                # 일반적으로 r의 2배 설정 (16*2=32)
                                # 높을수록 LoRA의 영향력 증가
    
    lora_dropout=0.05,          # 드롭아웃 비율: 과적합 방지
                                # 0.05 = 5% 드롭아웃, 일반적으로 0.05~0.1 사용
    
    # 적용할 모듈 지정 (Transformer의 주요 선형 레이어들)
    target_modules=[
        "q_proj",               # Query 프로젝션 레이어
        "k_proj",               # Key 프로젝션 레이어  
        "v_proj",               # Value 프로젝션 레이어
        "o_proj",               # Output 프로젝션 레이어
        "gate_proj",            # Gate 프로젝션 레이어 (FFN)
        "up_proj",              # Up 프로젝션 레이어 (FFN)
        "down_proj"             # Down 프로젝션 레이어 (FFN)
    ],
    
    task_type="CAUSAL_LM"       # 태스크 타입: 인과적 언어모델 (다음 토큰 예측)
)

model = get_peft_model(base_model, peft_config)

In [11]:
# 개선된 훈련 설정
tokenizer.model_max_length = 2048

cfg = SFTConfig(
    output_dir=f"/workspace/models/{selected_model}_qlora",
    
    # 훈련 스케줄링 - 본격 학습용 설정
    num_train_epochs=1,           # 테스트 1, 학습 5
    per_device_train_batch_size=4, # GPU 메모리 활용 최적화
    gradient_accumulation_steps=16, # 실제 배치 크기 = 4×16=64
    
    # 학습률 및 최적화
    learning_rate=1e-4,           # 안정적인 학습률
    weight_decay=0.01,            # 과적합 방지
    max_grad_norm=1.0,            # 그래디언트 클리핑
    
    # 로깅 및 저장 (개선된 주기)
    logging_steps=5,             
    logging_strategy="steps",
    logging_first_step=True,
    save_steps=250,               # 250스텝마다 저장
    save_total_limit=3,           # 최대 3개 체크포인트 유지
    
    # 평가 설정
    eval_strategy="steps",
    eval_steps=250,               # 250스텝마다 평가
    load_best_model_at_end=True,  # 최고 성능 모델 자동 선택
    metric_for_best_model="eval_loss",
    
    # 하드웨어 최적화
    bf16=True,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    
    # 학습률 스케줄러
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    
    # 메모리 최적화
    gradient_checkpointing=True,
    
    # 기타 설정
    remove_unused_columns=False,
    report_to=[],                 # wandb 등 비활성화
    # disable_tqdm=True,            # tqdm이 stdout 삼키는 이슈 회피
)

print("✅ 훈련 설정 완료!")
print(f"총 에포크: {cfg.num_train_epochs}")
print(f"실제 배치 크기: {cfg.per_device_train_batch_size} × {cfg.gradient_accumulation_steps} = {cfg.per_device_train_batch_size * cfg.gradient_accumulation_steps}")
print(f"로깅 주기: {cfg.logging_steps}스텝마다")


✅ 훈련 설정 완료!
총 에포크: 1
실제 배치 크기: 4 × 16 = 64
로깅 주기: 5스텝마다


In [12]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,   
    train_dataset=train_ds,       
    eval_dataset=eval_ds,
    args=cfg,
    data_collator=None,
)

Adding EOS to train dataset:   0%|          | 0/95487 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/95487 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/95487 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/5026 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/5026 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5026 [00:00<?, ? examples/s]

In [13]:
# Custom Callback

from transformers import TrainerCallback
from datetime import datetime
import pytz

class CustomCallback(TrainerCallback):
    tz_seoul = pytz.timezone("Asia/Seoul")

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            now = datetime.now(self.tz_seoul).strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{now}] step {state.global_step}\tloss {logs['loss']:.4f}", flush=True)

trainer.add_callback(CustomCallback())

In [ ]:
print("🎯 훈련 시작...")
trainer.train()

# 어댑터 저장
adapter_dir = f"/workspace/models/{selected_model}_qlora/adapter"
os.makedirs(adapter_dir, exist_ok=True)

trainer.model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)

print(f"✅ 어댑터 저장 완료: {adapter_dir}")
print(f"훈련 완료 후 GPU 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

🎯 훈련 시작...


Step,Training Loss,Validation Loss
250,1.812900,1.863725


[2025-08-09 21:45:01] step 1	loss 1.8448
[2025-08-09 21:46:28] step 5	loss 1.7969
[2025-08-09 21:48:20] step 10	loss 1.8193
[2025-08-09 21:50:11] step 15	loss 1.8288
[2025-08-09 21:52:00] step 20	loss 1.8326
[2025-08-09 21:53:46] step 25	loss 1.8313
[2025-08-09 21:55:43] step 30	loss 1.8230
[2025-08-09 21:57:36] step 35	loss 1.8267
[2025-08-09 21:59:31] step 40	loss 1.8520
[2025-08-09 22:01:22] step 45	loss 1.7723
[2025-08-09 22:03:14] step 50	loss 1.8260
[2025-08-09 22:05:08] step 55	loss 1.8002
[2025-08-09 22:07:00] step 60	loss 1.7889
[2025-08-09 22:08:53] step 65	loss 1.7753
[2025-08-09 22:10:47] step 70	loss 1.7932
[2025-08-09 22:12:42] step 75	loss 1.8160
[2025-08-09 22:14:35] step 80	loss 1.8175
[2025-08-09 22:16:29] step 85	loss 1.8094
[2025-08-09 22:18:26] step 90	loss 1.8297
[2025-08-09 22:20:16] step 95	loss 1.7721
[2025-08-09 22:22:10] step 100	loss 1.8281
[2025-08-09 22:24:05] step 105	loss 1.8060
[2025-08-09 22:25:57] step 110	loss 1.8526
[2025-08-09 22:27:53] step 115	lo

In [ ]:
# 훈련 완료 후 어댑터 병합 (선택사항)
# 훈련이 완료되면 이 셀을 실행하여 최종 모델을 생성하세요

def merge_adapter():
    """어댑터를 베이스 모델과 병합하여 최종 모델 생성"""
    
    adapter_dir = f"/workspace/models/{selected_model}_qlora/adapter"
    merged_dir = f"/workspace/models/{selected_model}_qlora/merged"
    
    print("🔄 어댑터 병합 시작...")
    
    # 메모리 정리
    import gc
    if 'model' in globals(): del model
    if 'base_model' in globals(): del base_model  
    if 'trainer' in globals(): del trainer
    torch.cuda.empty_cache()
    gc.collect()
    
    # 베이스 모델 재로딩 (풀프리시전)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        attn_implementation="eager"
    )
    
    # 어댑터 병합
    model = PeftModel.from_pretrained(model, adapter_dir)
    model = model.merge_and_unload()
    
    # 저장
    model.save_pretrained(merged_dir)
    tokenizer.save_pretrained(merged_dir)
    
    print(f"✅ 최종 모델 저장 완료: {merged_dir}")
    return merged_dir

# 사용법: merge_adapter() 함수를 호출하여 실행
print("💡 훈련 완료 후 merge_adapter() 함수를 실행하세요")


In [ ]:
merge_adapter()